<a href="https://colab.research.google.com/github/41371232H/PL_Repo/blob/main/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
from google.colab import userdata

api_key = userdata.get('gemini')
url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"

headers = {
    'Content-Type': 'application/json',
    'X-goog-api-key': api_key
}

data = {
    "contents": [
        {
            "parts": [
                {
                    "text": "Explain how AI works in a few words"
                }
            ]
        }
    ]
}

response = requests.post(url, headers=headers, data=json.dumps(data))

# 檢查回應狀態
if response.status_code == 200:
    # 成功回應，列印結果
    print(response.json())
    print("取得金鑰成功")
else:
    # 失敗回應，列印錯誤訊息
    print(f"Error: {response.status_code}")
    print(response.text)

{'candidates': [{'content': {'parts': [{'text': 'AI learns patterns from data to make predictions or decisions.\n'}], 'role': 'model'}, 'finishReason': 'STOP', 'avgLogprobs': -0.052846292654673256}], 'usageMetadata': {'promptTokenCount': 8, 'candidatesTokenCount': 12, 'totalTokenCount': 20, 'promptTokensDetails': [{'modality': 'TEXT', 'tokenCount': 8}], 'candidatesTokensDetails': [{'modality': 'TEXT', 'tokenCount': 12}]}, 'modelVersion': 'gemini-2.0-flash', 'responseId': 'pajUaKmkGKKhjrEPst790AY'}
取得金鑰成功


In [ ]:
import datetime
import gspread
import pandas as pd
import gradio as gr
from google.colab import auth
from google.auth import default

# Google 驗證
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 連線到 Google Sheet
gsheets = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/1rl14qyMAdt43zBnM4zcte6lQtqhbnGrXuHIg_8Kt8C8/edit?gid=0#gid=0'
)

categories = ["食", "衣", "住", "行", "樂", "其他"]

# 建立工作表
try:
    plan_sheet = gsheets.worksheet('計劃')
except:
    plan_sheet = gsheets.add_worksheet(title="計劃", rows="100", cols="10")
    plan_sheet.append_row(["生活費"] + categories)

try:
    record_sheet = gsheets.worksheet('紀錄')
except:
    record_sheet = gsheets.add_worksheet(title="紀錄", rows="1000", cols="10")
    record_sheet.append_row(["日期", "分類", "品項", "金額", "總金額"])


# ========= 計劃檢查與顯示 =========
def get_current_plan():
    plan_rows = plan_sheet.get_all_values()
    if len(plan_rows) <= 1:
        return "⚠️ 尚無計劃，請先設定！"
    else:
        df_plan = pd.DataFrame(plan_rows[1:], columns=plan_rows[0])
        return df_plan.to_string(index=False)

def has_plan():
    plan_rows = plan_sheet.get_all_values()
    return len(plan_rows) > 1


# ========= 修改計劃 =========
def modify_plan(budget, 食, 衣, 住, 行, 樂, 其他):
    allocations = {"食": 食, "衣": 衣, "住": 住, "行": 行, "樂": 樂, "其他": 其他}
    total_alloc = sum(allocations.values())
    if total_alloc != budget:
        return f"⚠️ 分配總和 {total_alloc} 不等於生活費 {budget}，請重新輸入！", get_current_plan(), [None]*7

    plan_sheet.clear()
    plan_sheet.append_row(["生活費"] + categories)
    plan_sheet.append_row([budget] + [allocations[c] for c in categories])
    return "✅ 計劃已更新！", get_current_plan(), [None]*7


# ========= 新增紀錄 =========
def add_record(date_str, category, item, amount):
    if not has_plan():
        return "⚠️ 尚無計劃，請先到『修改計劃』設定！", "", "", "", ""

    try:
        date_obj = datetime.datetime.strptime(date_str, "%Y-%m-%d")
        date_fmt = date_obj.strftime("%Y/%m/%d")
    except:
        return "⚠️ 日期格式錯誤 (應為 YYYY-MM-DD)", "", "", "", ""

    try:
        amount = float(amount)
    except:
        return "⚠️ 金額格式錯誤", "", "", "", ""

    rows = record_sheet.get_all_values()
    if len(rows) > 1 and rows[-1][4] != "":
        total_spent = float(rows[-1][4])
    else:
        total_spent = 0.0

    total_spent += amount
    record_sheet.append_row(
        [date_fmt, category, item, amount, total_spent],
        value_input_option="USER_ENTERED"
    )

    return "✅ 已新增紀錄！", "", "", "", "", query_status()


# ========= 刪除紀錄 =========
def delete_record(idx):
    rows = record_sheet.get_all_values()
    if len(rows) <= 1:
        return "⚠️ 尚無任何紀錄", show_records()

    df = pd.DataFrame(rows[1:], columns=rows[0])
    try:
        idx = int(idx)
        if idx < 0 or idx >= len(df):
            return "⚠️ 編號超出範圍", show_records()
    except:
        return "⚠️ 請輸入數字", show_records()

    row_num = idx + 2
    record_sheet.delete_rows(row_num)

    # 重算總金額
    rows = record_sheet.get_all_values()
    if len(rows) > 1:
        df = pd.DataFrame(rows[1:], columns=rows[0])
        df['金額'] = pd.to_numeric(df['金額'], errors='coerce')
        df['總金額'] = df['金額'].cumsum()
        record_sheet.clear()
        record_sheet.append_row(["日期", "分類", "品項", "金額", "總金額"])
        record_sheet.update(f"A2:E{len(df)+1}", df.values.tolist())

    return f"✅ 已刪除第 {idx} 筆紀錄！", show_records()


def show_records():
    rows = record_sheet.get_all_values()
    if len(rows) <= 1:
        return "⚠️ 尚無任何紀錄"
    df = pd.DataFrame(rows[1:], columns=rows[0])
    df.index = range(len(df))  # 讓 index 成為紀錄編號
    return df.to_string()


# ========= 查詢支出 =========
def query_status():
    if not has_plan():
        return "⚠️ 尚無計劃，請先到『修改計劃』設定！"

    rows = record_sheet.get_all_values()
    if len(rows) <= 1:
        return "⚠️ 尚無任何紀錄"

    df = pd.DataFrame(rows[1:], columns=rows[0])
    df["金額"] = pd.to_numeric(df["金額"], errors="coerce")

    total_spent = df["金額"].sum()
    by_category = df.groupby("分類")["金額"].sum().to_dict()

    output = f"📊 總支出：{total_spent:.2f}\n"
    for c in categories:
        spent = by_category.get(c, 0)
        output += f"{c}：{spent:.2f}\n"

    return output


# ========= 結算報告 =========
def generate_report():
    if not has_plan():
        return "⚠️ 尚無計劃，請先設定！"

    plan_rows = plan_sheet.get_all_values()
    plan = list(map(float, plan_rows[-1][1:]))
    df_plan = dict(zip(categories, plan))

    rows = record_sheet.get_all_values()
    if len(rows) <= 1:
        return "⚠️ 尚無任何紀錄"

    df = pd.DataFrame(rows[1:], columns=rows[0])
    df['金額'] = pd.to_numeric(df['金額'], errors='coerce')

    by_category = df.groupby("分類")['金額'].sum()
    total_spent = df['金額'].sum()
    total_budget = sum(df_plan.values())
    total_remain = total_budget - total_spent

    report = "📊 結算報告\n\n"
    category_status = {}
    for c in categories:
        spent = by_category[c] if c in by_category else 0
        remain = df_plan[c] - spent
        category_status[c] = {"spent": spent, "remain": remain}
        if remain < -0.15 * df_plan[c]:
            slogan = "嚴重透支"
        elif abs(remain) <= 0.15 * df_plan[c]:
            slogan = "收支平衡"
        else:
            slogan = "有積蓄"
        report += f"{c}: 花費 {spent:.2f}, 剩餘 {remain:.2f}, {slogan}\n"

    over = sum(1 for c in categories if category_status[c]['remain'] < 0)
    under = sum(1 for c in categories if category_status[c]['remain'] > 0)

    if over == 0:
        feedback = "全部分類都有積蓄，很棒哦！"
    elif over > 0 and under > 0 and total_remain >= 0:
        feedback = "部分分類超支，但總體還有剩餘，建議檢討超支的部分哦!"
    elif over > 0 and under > 0 and total_remain < 0:
        feedback = "部分分類超支且總體透支，可能需要重新規劃哦！"
    elif under >= len(categories) // 2:
        feedback = "大部分分類都有積蓄，維持目前策略就可以啦!"
    elif over >= len(categories) // 2:
        feedback = "大部分分類超支，建議控制支出或重新分配預算哦!"
    else:
        feedback = "請注意收支情況，持續追蹤！"

    report += f"\n總花費: {total_spent:.2f}, 總剩餘: {total_remain:.2f}\n"
    report += f"\n🔔 整體回饋：{feedback}"

    return report


# ========= Gradio 介面 =========
with gr.Blocks() as demo:
    gr.Markdown("# 💰 Google Sheet 記帳系統")

    # ---------- 修改計劃 ----------
    with gr.Tab("修改計劃"):
        current_plan = gr.Textbox(label="目前計劃", lines=6)
        budget = gr.Number(label="生活費總金額")
        inputs = [gr.Number(label=c) for c in categories]
        plan_btn = gr.Button("更新計劃")
        plan_output = gr.Textbox(label="結果")

        plan_btn.click(modify_plan, inputs=[budget]+inputs, outputs=[plan_output, current_plan]+inputs)
        demo.load(get_current_plan, inputs=None, outputs=current_plan)

    # ---------- 新增紀錄 ----------
    with gr.Tab("新增紀錄"):
        date_input = gr.Textbox(label="日期 (YYYY-MM-DD)")
        category_input = gr.Dropdown(categories, label="分類")
        item_input = gr.Textbox(label="品項")
        amount_input = gr.Number(label="金額")
        add_btn = gr.Button("新增")
        add_output = gr.Textbox(label="結果")
        status_output = gr.Textbox(label="目前支出狀況", lines=12)

        add_btn.click(add_record,
                      inputs=[date_input, category_input, item_input, amount_input],
                      outputs=[add_output, date_input, category_input, item_input, amount_input, status_output])

    # ---------- 刪除紀錄 ----------
    with gr.Tab("刪除紀錄"):
        records_output2 = gr.Textbox(label="所有紀錄", lines=12)

        # 🔹 先放刷新按鈕
        refresh_btn = gr.Button("刷新紀錄")
        refresh_btn.click(show_records, inputs=None, outputs=records_output2)

        # 🔹 再放輸入要刪除的紀錄編號
        idx_input = gr.Textbox(label="要刪除的紀錄編號")

        delete_btn = gr.Button("刪除")
        delete_output = gr.Textbox(label="結果")

        # 刪除後自動刷新紀錄
        delete_btn.click(delete_record, inputs=idx_input, outputs=[delete_output, records_output2])

    # ---------- 查詢支出 ----------
    with gr.Tab("查詢支出"):
        query_output = gr.Textbox(label="支出情況", lines=12)
        query_btn = gr.Button("查詢")
        query_btn.click(query_status, inputs=[], outputs=query_output)

    # ---------- 結算報告 ----------
    with gr.Tab("結算報告"):
        report_output = gr.Textbox(label="結算報告", lines=12)
        report_btn = gr.Button("生成報告")
        report_btn.click(generate_report, inputs=[], outputs=report_output)

demo.launch()


### **最終程式碼**

In [ ]:
import datetime
import gspread
import pandas as pd
import gradio as gr
from google.colab import auth
from google.auth import default

import requests
import json
from google.colab import userdata

api_key = userdata.get('gemini')
url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"

headers = {
    'Content-Type': 'application/json',
    'X-goog-api-key': api_key
}

data = {
    "contents": [
        {
            "parts": [
                {
                    "text": "Explain how AI works in a few words"
                }
            ]
        }
    ]
}

response = requests.post(url, headers=headers, data=json.dumps(data))

# Google 驗證
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 連線到 Google Sheet
gsheets = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/1rl14qyMAdt43zBnM4zcte6lQtqhbnGrXuHIg_8Kt8C8/edit?gid=0#gid=0'
)

In [1]:
import datetime
import gspread
import pandas as pd
import gradio as gr
import requests
import json
from google.colab import auth, userdata
from google.auth import default

# Google 驗證
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 連線到 Google Sheet
gsheets = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/1W0EMUIir8dF3LhCMafo8CRo1B_m_0EB7YRMpLvO5Ru8/edit?usp=sharing'
)

categories = ["食", "衣", "住", "行", "樂", "其他"]

# 建立工作表
try:
    plan_sheet = gsheets.worksheet('計劃')
except:
    plan_sheet = gsheets.add_worksheet(title="計劃", rows="100", cols="10")
    plan_sheet.append_row(["生活費"] + categories)

try:
    record_sheet = gsheets.worksheet('紀錄')
except:
    record_sheet = gsheets.add_worksheet(title="紀錄", rows="1000", cols="10")
    record_sheet.append_row(["日期", "分類", "品項", "金額", "總金額"])


# ========= 計劃檢查與顯示 =========
def get_current_plan():
    plan_rows = plan_sheet.get_all_values()
    if len(plan_rows) <= 1:
        return "⚠️ 尚無計劃，請先設定！"
    else:
        df_plan = pd.DataFrame(plan_rows[1:], columns=plan_rows[0])
        return df_plan.to_string(index=False)

def has_plan():
    plan_rows = plan_sheet.get_all_values()
    return len(plan_rows) > 1


# ========= 修改計劃 =========
def modify_plan(budget, 食, 衣, 住, 行, 樂, 其他):
    allocations = {"食": 食, "衣": 衣, "住": 住, "行": 行, "樂": 樂, "其他": 其他}
    total_alloc = sum(allocations.values())
    if total_alloc != budget:
        return f"⚠️ 分配總和 {total_alloc} 不等於生活費 {budget}，請重新輸入！", get_current_plan(), [None]*7

    plan_sheet.clear()
    plan_sheet.append_row(["生活費"] + categories)
    plan_sheet.append_row([budget] + [allocations[c] for c in categories])
    return "✅ 計劃已更新！", get_current_plan(), [None]*7


# ========= 新增紀錄 =========
def add_record(date_str, category, item, amount):
    if not has_plan():
        return "⚠️ 尚無計劃，請先到『修改計劃』設定！", "", "", "", ""

    try:
        date_obj = datetime.datetime.strptime(date_str, "%Y-%m-%d")
        date_fmt = date_obj.strftime("%Y/%m/%d")
    except:
        return "⚠️ 日期格式錯誤 (應為 YYYY-MM-DD)", "", "", "", ""

    try:
        amount = float(amount)
    except:
        return "⚠️ 金額格式錯誤", "", "", "", ""

    rows = record_sheet.get_all_values()
    if len(rows) > 1 and rows[-1][4] != "":
        total_spent = float(rows[-1][4])
    else:
        total_spent = 0.0

    total_spent += amount
    record_sheet.append_row(
        [date_fmt, category, item, amount, total_spent],
        value_input_option="USER_ENTERED"
    )

    return "✅ 已新增紀錄！", "", "", "", "", query_status()


# ========= 刪除紀錄 =========
def delete_record(idx):
    rows = record_sheet.get_all_values()
    if len(rows) <= 1:
        return "⚠️ 尚無任何紀錄", show_records()

    df = pd.DataFrame(rows[1:], columns=rows[0])
    try:
        idx = int(idx)
        if idx < 0 or idx >= len(df):
            return "⚠️ 編號超出範圍", show_records()
    except:
        return "⚠️ 請輸入數字", show_records()

    row_num = idx + 2
    record_sheet.delete_rows(row_num)

    # 重算總金額
    rows = record_sheet.get_all_values()
    if len(rows) > 1:
        df = pd.DataFrame(rows[1:], columns=rows[0])
        df['金額'] = pd.to_numeric(df['金額'], errors='coerce')
        df['總金額'] = df['金額'].cumsum()
        record_sheet.clear()
        record_sheet.append_row(["日期", "分類", "品項", "金額", "總金額"])
        record_sheet.update(f"A2:E{len(df)+1}", df.values.tolist())

    return f"✅ 已刪除第 {idx} 筆紀錄！", show_records()


def show_records():
    rows = record_sheet.get_all_values()
    if len(rows) <= 1:
        return "⚠️ 尚無任何紀錄"
    df = pd.DataFrame(rows[1:], columns=rows[0])
    df.index = range(len(df))  # 讓 index 成為紀錄編號
    return df.to_string()


# ========= 查詢支出 =========
def query_status():
    if not has_plan():
        return "⚠️ 尚無計劃，請先到『修改計劃』設定！"

    rows = record_sheet.get_all_values()
    if len(rows) <= 1:
        return "⚠️ 尚無任何紀錄"

    df = pd.DataFrame(rows[1:], columns=rows[0])
    df["金額"] = pd.to_numeric(df["金額"], errors="coerce")

    total_spent = df["金額"].sum()
    by_category = df.groupby("分類")["金額"].sum().to_dict()

    output = f"📊 總支出：{total_spent:.2f}\n"
    for c in categories:
        spent = by_category.get(c, 0)
        output += f"{c}：{spent:.2f}\n"

    return output


# ========= 結算報告 =========
def generate_report():
    if not has_plan():
        return "⚠️ 尚無計劃，請先設定！"

    plan_rows = plan_sheet.get_all_values()
    plan = list(map(float, plan_rows[-1][1:]))
    df_plan = dict(zip(categories, plan))

    rows = record_sheet.get_all_values()
    if len(rows) <= 1:
        return "⚠️ 尚無任何紀錄"

    df = pd.DataFrame(rows[1:], columns=rows[0])
    df['金額'] = pd.to_numeric(df['金額'], errors='coerce')

    by_category = df.groupby("分類")['金額'].sum()
    total_spent = df['金額'].sum()
    total_budget = sum(df_plan.values())
    total_remain = total_budget - total_spent

    report = "📊 結算報告\n\n"
    category_status = {}
    for c in categories:
        spent = by_category[c] if c in by_category else 0
        remain = df_plan[c] - spent
        category_status[c] = {"spent": spent, "remain": remain}
        if remain < -0.15 * df_plan[c]:
            slogan = "嚴重透支"
        elif abs(remain) <= 0.15 * df_plan[c]:
            slogan = "收支平衡"
        else:
            slogan = "有積蓄"
        report += f"{c}: 花費 {spent:.2f}, 剩餘 {remain:.2f}, {slogan}\n"

    over = sum(1 for c in categories if category_status[c]['remain'] < 0)
    under = sum(1 for c in categories if category_status[c]['remain'] > 0)

    if over == 0:
        feedback = "全部分類都有積蓄，很棒哦！"
    elif over > 0 and under > 0 and total_remain >= 0:
        feedback = "部分分類超支，但總體還有剩餘，建議檢討超支的部分哦!"
    elif over > 0 and under > 0 and total_remain < 0:
        feedback = "部分分類超支且總體透支，可能需要重新規劃哦！"
    elif under >= len(categories) // 2:
        feedback = "大部分分類都有積蓄，維持目前策略就可以啦!"
    elif over >= len(categories) // 2:
        feedback = "大部分分類超支，建議控制支出或重新分配預算哦!"
    else:
        feedback = "請注意收支情況，持續追蹤！"

    report += f"\n總花費: {total_spent:.2f}, 總剩餘: {total_remain:.2f}\n"
    report += f"\n🔔 整體回饋：{feedback}"

    return report


# ========= AI 諮詢服務 =========
api_key = userdata.get('gemini')
url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"
headers = {
    'Content-Type': 'application/json',
    'X-goog-api-key': api_key
}

def analyze_with_ai(report_text):
    if not report_text or report_text.startswith("⚠️"):
        return "⚠️ 目前沒有結算報告可分析"

    data = {
        "contents": [
            {
                "parts": [
                    {
                        "text": f"以下是一份個人記帳的結算報告，請用專業理財顧問的角度，給我分析與建議：\n\n{report_text}"
                    }
                ]
            }
        ]
    }
    try:
        response = requests.post(url, headers=headers, data=json.dumps(data))
        result = response.json()
        return result['candidates'][0]['content']['parts'][0]['text']
    except Exception as e:
        return f"❌ AI 分析失敗：{str(e)}"


# ========= Gradio 介面 =========
with gr.Blocks() as demo:
    gr.Markdown("# 💰 Google Sheet 記帳系統 + AI 理財分析")

    # ---------- 修改計劃 ----------
    with gr.Tab("修改計劃"):
        current_plan = gr.Textbox(label="目前計劃", lines=6)
        budget = gr.Number(label="生活費總金額")
        inputs = [gr.Number(label=c) for c in categories]
        plan_btn = gr.Button("更新計劃")
        plan_output = gr.Textbox(label="結果")

        plan_btn.click(modify_plan, inputs=[budget]+inputs, outputs=[plan_output, current_plan]+inputs)
        demo.load(get_current_plan, inputs=None, outputs=current_plan)

    # ---------- 新增紀錄 ----------
    with gr.Tab("新增紀錄"):
        date_input = gr.Textbox(label="日期 (YYYY-MM-DD)")
        category_input = gr.Dropdown(categories, label="分類")
        item_input = gr.Textbox(label="品項")
        amount_input = gr.Number(label="金額")
        add_btn = gr.Button("新增")
        add_output = gr.Textbox(label="結果")
        status_output = gr.Textbox(label="目前支出狀況", lines=12)

        add_btn.click(add_record,
                      inputs=[date_input, category_input, item_input, amount_input],
                      outputs=[add_output, date_input, category_input, item_input, amount_input, status_output])

    # ---------- 刪除紀錄 ----------
    with gr.Tab("刪除紀錄"):
        records_output2 = gr.Textbox(label="所有紀錄", lines=12)

        # 🔹 刷新按鈕
        refresh_btn = gr.Button("刷新紀錄")
        refresh_btn.click(show_records, inputs=None, outputs=records_output2)

        # 🔹 輸入刪除編號
        idx_input = gr.Textbox(label="要刪除的紀錄編號")
        delete_btn = gr.Button("刪除")
        delete_output = gr.Textbox(label="結果")

        delete_btn.click(delete_record, inputs=idx_input, outputs=[delete_output, records_output2])

    # ---------- 查詢支出 ----------
    with gr.Tab("查詢支出"):
        query_output = gr.Textbox(label="支出情況", lines=12)
        query_btn = gr.Button("查詢")
        query_btn.click(query_status, inputs=[], outputs=query_output)

    # ---------- 結算報告 + AI ----------
    with gr.Tab("結算報告"):
        report_output = gr.Textbox(label="結算報告", lines=12)
        report_btn = gr.Button("生成報告")
        ai_btn = gr.Button("AI 分析")
        ai_output = gr.Textbox(label="AI 理財顧問建議", lines=12)

        report_btn.click(generate_report, inputs=[], outputs=report_output)
        ai_btn.click(analyze_with_ai, inputs=report_output, outputs=ai_output)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0184f79a274c99dceb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
